<a href="https://colab.research.google.com/github/AmirHesamKamalpour/Query-search-Information-Retrieval-using-Inverted-Index/blob/main/Information_Retrieval_Inverted_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***AmirHesam Kamalpour***

# **Not important parts**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amirhesamkamalpour","key":"a238553a8e0e2588be04a05252f89979"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 93% 24.0M/25.7M [00:01<00:00, 32.7MB/s]
100% 25.7M/25.7M [00:01<00:00, 21.4MB/s]


In [ ]:
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction import text
from collections import defaultdict
from enum import Enum

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')

# **Kaggle dataset: movie-reviews**

In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
df.drop(columns= 'sentiment', inplace = True)

In [ ]:
df

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...
49995,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...


In [ ]:
def choose_n_random_documents(dataset, n : int):
  return dataset.sample(n = n).reset_index(drop=True)

In [ ]:
new_df = choose_n_random_documents(df, 10)

In [ ]:
new_df

,review
0,I rented this film to see what might be a bloo...
1,"First of all, this plot is way overdone - girl..."
2,This crock of doodoo won a award? They must ha...
3,American Pie has gone a long distance from the...
4,Seth McFarlane is a true genius. He has crafte...
5,I cannot stress how bad this movie is. This di...
6,This really is the worst film I have ever seen...
7,I think it's incredibly hard to write any kind...
8,White man + progress + industrialization = BAD...
9,"Eric Rohmer's ""The Lady and the Duke"". could h..."


# **Removing Punctuation**

In [ ]:
def remove_punctuation(dataset, column : string):
  dataset[column] = dataset[column].str.replace('[{}]'.format(string.punctuation), '')
  return dataset

In [ ]:
new_df = remove_punctuation(new_df, 'review')

<ipython-input-107-cd4f82f2837a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[column] = dataset[column].str.replace('[{}]'.format(string.punctuation), '')


In [ ]:
new_df

,review
0,I rented this film to see what might be a bloo...
1,First of all this plot is way overdone girl w...
2,This crock of doodoo won a award They must hav...
3,American Pie has gone a long distance from the...
4,Seth McFarlane is a true genius He has crafted...
5,I cannot stress how bad this movie is This dir...
6,This really is the worst film I have ever seen...
7,I think its incredibly hard to write any kind ...
8,White man progress industrialization BAD Fi...
9,Eric Rohmers The Lady and the Duke could have ...


# **Lowercase Documnets**

In [ ]:
new_df['review'] = new_df['review'].str.lower()

# **Removing Stopwords**

In [ ]:
def remove_stop_words(dataset, column: string):
  stop_words = text.ENGLISH_STOP_WORDS
  pat = r'\b(?:{})\b'.format('|'.join(stop_words))
  dataset[column] = dataset[column].str.replace(pat, '')
  dataset[column] = dataset[column].str.replace(r'\s+', ' ')
  return dataset

In [ ]:
new_df = remove_stop_words(new_df, 'review')

<ipython-input-111-af13035880ac>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[column] = dataset[column].str.replace(pat, '')
<ipython-input-111-af13035880ac>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[column] = dataset[column].str.replace(r'\s+', ' ')


In [ ]:
new_df

,review
0,rented film bloody non stop action movie got ...
1,plot way overdone girl wants make loves snobb...
2,crock doodoo won award desperate giving award...
3,american pie gone long distance believe actors...
4,seth mcfarlane true genius crafted witty cultu...
5,stress bad movie director took cheap little u...
6,really worst film seen period actually paid £...
7,think incredibly hard write kind fullscale re...
8,white man progress industrialization bad natio...
9,eric rohmers lady duke used better translation...


# **Build the inverted index**


In [ ]:
def sort_inverted_index(inverted_index):
    sorted_index = sorted(inverted_index.items(), key=lambda x: x[0])
    sorted_index = dict(sorted_index)
    return sorted_index

In [ ]:
def build_inverted_index(dataset, column: string):
  inverted_index = defaultdict(set)
  for index, row in dataset.iterrows():
    document_id = index
    words = row[column].split()
    for word in words:
        inverted_index[word].add(document_id)
  inverted_index = sort_inverted_index(inverted_index)
  return inverted_index

In [ ]:
inverted_index = build_inverted_index(new_df, 'review')

In [ ]:
inverted_index

{'1': {2},
 '10': {2},
 '100': {6},
 '110': {1, 6},
 '183': {7},
 'able': {9},
 'absolute': {3},
 'accurate': {9},
 'achieved': {7},
 'acted': {6},
 'acting': {0, 2, 7},
 'action': {0},
 'actiondrama': {0},
 'actors': {3, 5},
 'actress': {1},
 'actually': {6, 7, 8},
 'againbr': {6},
 'agitators': {9},
 'agree': {7},
 'allbr': {1},
 'almighty': {3},
 'alongside': {2},
 'amazing': {1},
 'america': {8},
 'american': {3},
 'animals': {8},
 'animation': {8},
 'answers': {6},
 'appeals': {8},
 'appears': {9},
 'apple': {3},
 'apposite': {7},
 'arent': {4},
 'aristocrats': {9},
 'aroundbr': {9},
 'art': {7, 9},
 'ask': {5},
 'astonishingly': {7},
 'atmospheric': {7},
 'attack': {8},
 'attending': {3},
 'attention': {9},
 'attractive': {1, 7},
 'award': {2},
 'away': {0, 3},
 'backyard': {0},
 'bad': {0, 2, 5, 8},
 'banned': {5},
 'bar': {0},
 'bearable': {2},
 'beauty': {7},
 'behavior': {7},
 'believe': {2, 3, 9},
 'belting': {1},
 'better': {3, 7, 9},
 'black': {7},
 'blackringed': {7},
 'b

In [ ]:
sorted_items = sorted(inverted_index.items(), key=lambda x: len(x[1]), reverse = True)

for key, value in sorted_items:
    print(key, value)

movie {0, 1, 2, 5, 6, 7, 8, 9}
br {1, 2, 3, 4, 5, 6, 9}
bad {0, 8, 2, 5}
cast {9, 5, 6, 7}
dont {1, 3, 6, 7}
good {0, 1, 8, 9}
think {8, 1, 4, 7}
way {8, 1, 3, 6}
acting {0, 2, 7}
actually {8, 6, 7}
believe {9, 2, 3}
better {9, 3, 7}
especially {8, 1, 7}
fact {8, 1, 3}
film {0, 9, 6}
gets {0, 9, 6}
head {0, 9, 5}
just {1, 3, 4}
kind {8, 5, 7}
like {0, 6, 7}
long {3, 4, 7}
look {0, 5, 7}
lost {8, 1, 7}
make {0, 1, 5}
making {1, 5, 9}
man {8, 9, 7}
movies {8, 5, 7}
people {9, 5, 7}
point {1, 6, 7}
scene {0, 6, 7}
shot {0, 8, 5}
time {3, 6, 7}
true {9, 4, 7}
watch {9, 2, 6}
110 {1, 6}
actors {3, 5}
art {9, 7}
attractive {1, 7}
away {0, 3}
characters {0, 7}
cheap {0, 5}
clever {4, 5}
course {3, 7}
days {8, 9}
director {2, 5}
does {8, 9}
enjoy {3, 7}
excellent {9, 7}
form {9, 7}
free {8, 3}
got {0, 6}
guy {0, 4}
hard {0, 7}
horrid {0, 2}
im {1, 3}
incredibly {6, 7}
looks {1, 7}
makes {0, 4}
music {0, 1}
original {8, 3}
piece {9, 2}
plot {1, 3}
real {5, 7}
really {1, 6}
remains {9, 7}
rememb

In [ ]:
inverted_index

# **Search Queries**

In [ ]:
class operators(Enum):
  AND = 'and'
  OR = 'or'
  NOT = 'not'

In [ ]:
def search(query, inverted_index):
    query_terms = query.split()
    dif = set([0,1,2,3,4,5,6,7,8,9])
    relevant_docs = set()
    i = 0
    while i < len(query_terms):

        term = query_terms[i]

        match term:

          case operators.AND.value:

            if query_terms[i+1] == 'not':
              x = inverted_index[query_terms[i+2]]
              y = dif.copy()
              y.difference_update(x)
              relevant_docs = relevant_docs.intersection(y)
              i+=3

            else:
              relevant_docs = relevant_docs.intersection(inverted_index[query_terms[i+1]])
              i+=2

          case operators.OR.value:

            if query_terms[i+1] == 'not':
              x = inverted_index[query_terms[i+2]]
              y = dif.copy()
              y.difference_update(x)
              relevant_docs = relevant_docs.union(y)
              i+=3

            else:
              relevant_docs = relevant_docs.union(inverted_index[query_terms[i+1]])
              i+=2

          case operators.NOT.value:

            if i == 0:
              x = inverted_index[query_terms[i+1]]
              y = dif.copy()
              y.difference_update(x)
              relevant_docs = relevant_docs.union(y)
              i+=2

          case default:

            try:
              relevant_docs = relevant_docs.union(inverted_index[term])
              i+=1

            except:
              print(f"the word: {term} is not in the documents")
              break


    return relevant_docs

In [ ]:
query = input("Enter your query: ").lower()
relevant_docs = search(query, inverted_index)
print(f"relevant document ids: {relevant_docs}")

Enter your query: movie and not got
relevant document ids: {1, 2, 5, 7, 8, 9}


# alternative

In [ ]:
# def search1(query, inverted_index):

#     query_terms = query.split()
#     dif = set([0,1,2,3,4])
#     relevant_docs = set()
#     i = 0

#     while i < len(query_terms):

#       if query_terms[i] == 'and':

#         if query_terms[i+1] == 'not':
#           x = inverted_index[query_terms[i+2]]
#           y = dif.copy()
#           y.difference_update(x)
#           relevant_docs = relevant_docs.intersection(y)
#           i+=3

#         else:
#           relevant_docs = relevant_docs.intersection(inverted_index[query_terms[i+1]])
#           i+=2

#       elif query_terms[i] == 'or':

#         if query_terms[i+1] == 'not':
#           x = inverted_index[query_terms[i+2]]
#           y = dif.copy()
#           y.difference_update(x)
#           relevant_docs = relevant_docs.union(y)
#           i+=3

#         else:
#           relevant_docs = relevant_docs.union(inverted_index[query_terms[i+1]])
#           i+=2

#       elif query_terms[0] == 'not':

#         x = inverted_index[query_terms[i+1]]
#         y = dif.copy()
#         y.difference_update(x)
#         relevant_docs = relevant_docs.union(y)
#         i+=2

#       else:
#         relevant_docs = relevant_docs.union(inverted_index[query_terms[i]])
#         i+=1


#     return relevant_docs